In [11]:
import numpy as np
import gzip
import os
import tensorflow as tf
import re

# specify the path to the zipped file.
os.chdir('/mnt/c/Users/amsj1/OneDrive - University of Tennessee/2nd_year/BZAN554_deep_learning/bzan554-group-assignment1')

In [12]:
def parse(path):
    """
    Function to read in the VERY LARGE dataset and yield it as a generator for
    memory efficiency. Takes one argument which is the path to the file being
    read in. This path is set abose using 'os.chdir'.
    """
    with gzip.open(path, 'rb') as g:
        for l in g:
            yield eval(l)

#######################################################################
# Basically load in some data to make lookup tables for the sequentially
# learning part
#######################################################################
# X_raw = []
lookup_X = []
Y_raw = []
counter = 0
for d in parse('meta_Clothing_Shoes_and_Jewelry.json.gz'):
    counter += 1
    # X_raw.append(d['title'])
    X_raw = np.array(d['title'])
    Y_raw.append(d['category'])

    X_words = [words for words in str(X_raw).lower().split()]
    lookup_X.extend(X_words)

    if counter == 100:
        break

# X_words = []
# for titles in X_raw:
#     # split the titles into words and lowercase
#     words = titles.lower().split()
#     for word in words:
#         # extract out words with minimal to no symbols
#         stripped_word = re.sub('[0-9,.,#,\-,&,;,\',(,),/,@,+,!]', '', word)
#         X_words.append(stripped_word)

# X_words = np.unique(np.array(X_words))

# look up for X
# lookup_X = []
# lookup_X.extend([word for word in X_words])

# flatten the Y list of lists
flat_Y = [category for subcat in Y_raw for category in subcat]
# uniquefy it
unique_Y = np.unique(np.array(flat_Y))

# look up for Y
lookup_Y = []
lookup_Y.extend([category for category in unique_Y])

In [13]:
#######################################################################
# Set up the model architecture
#######################################################################
inputs = tf.keras.layers.Input(shape = (len(lookup_X),))
hidden1 = tf.keras.layers.Dense(
    units = len(lookup_X),
    activation = 'sigmoid',
    name = 'hidden1'
)(inputs)
hidden2 = tf.keras.layers.Dense(
    units = len(lookup_Y),
    activation = 'sigmoid',
    name = 'hidden2'
)(hidden1)
output = tf.keras.layers.Dense(
    units = len(unique_Y),
    activation = 'sigmoid',
    name = 'output'
)(hidden2)

#######################################################################
# create & compile the model
#######################################################################
model = tf.keras.Model(inputs = inputs, outputs = output)
model.compile(
    loss = 'binary_crossentropy',
    optimizer = tf.keras.optimizers.SGD(learning_rate = 0.001)
)

In [14]:
#######################################################################
# Big boy model learning time
#######################################################################
for i in range(3): # number of epochs
    counter = 0
    for d in parse('meta_Clothing_Shoes_and_Jewelry.json.gz'): # instances
        counter += 1
        X_raw = np.array(d['title'])
        Y_raw = np.array(d['category'])

        ###############################################
        # logic for the X hot-one-encoded stuff
        ###############################################
        # split the titles into words and lowercase
        # words = word.lower().split()
        X_words = [words for words in str(X_raw).lower().split()]

        # Get final binary stuff for X
        X_indices = np.where(np.isin(lookup_X, X_words))
        X_final = np.zeros(len(lookup_X))
        X_final[X_indices] = 1
        X_final = X_final.reshape(1, len(X_final))

        ###############################################
        # logic for the Y hot-one-encoded stuff
        ###############################################
        Y_indices = np.where(np.isin(lookup_Y, unique_Y))
        Y_final = np.zeros(len(lookup_Y))
        Y_final[Y_indices] = 1
        Y_final = Y_final.reshape(1, len(Y_final))

        ###############################################
        # Finallllyyyyy fit the freakin model
        ###############################################
        model.fit(x=X_final,y=Y_final,epochs=1,batch_size=1)

        if counter == 100:
            break

1/1 [==============================] - 0s 9ms/step - loss: 0.7085


In [15]:
yhat = model.predict(x=X_final)
yhat

array([[0.68660796, 0.482992  , 0.48109397, 0.37659353, 0.42509547,
        0.2899843 , 0.56060725, 0.61238474, 0.502351  , 0.42144728,
        0.40341514, 0.27052984, 0.49330765, 0.52238065, 0.5881876 ,
        0.55427855, 0.26908696, 0.357403  , 0.60082215, 0.43504816,
        0.4661101 , 0.27060544, 0.5764306 , 0.57110906, 0.35426617,
        0.39707398, 0.521375  , 0.562667  , 0.41733658, 0.46671253,
        0.413778  , 0.4664705 , 0.6047629 , 0.46542802, 0.41201028,
        0.70283145, 0.4171144 , 0.51953036, 0.61058056, 0.4558437 ,
        0.64620394, 0.4029119 , 0.31916064, 0.40306658, 0.3581658 ,
        0.5209357 , 0.3916538 , 0.58427227, 0.518163  , 0.62125915,
        0.55668247, 0.4035258 , 0.62890285, 0.58375525, 0.42473564,
        0.6522109 , 0.6638233 , 0.61754704, 0.3657757 , 0.53198314,
        0.44234675, 0.5434642 , 0.54443944, 0.62236416, 0.69212687,
        0.4628481 , 0.35777912, 0.57708126, 0.46135515, 0.5019346 ,
        0.46750984, 0.4441597 , 0.6372826 , 0.64